In [147]:
import pandas as pd
import pyreadr
import numpy as np
from tqdm import tqdm

from unidecode import unidecode

In [148]:
def write_error(msg):
    with open('errors.txt', 'w') as f:
        f.write(msg)

In [149]:
path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito.Rdata'
pathvote = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito_vote.Rdata'

df = pyreadr.read_r(path)['cand']
vote1 = pyreadr.read_r(pathvote)['cand']

#vote1 = vote1[vote1.tipo_eleicao == 'eleicao ordinaria']

vote = vote1

In [150]:
#Only result for 2802601 in 2004 is nao eleito handled
#Only result for 2803609 in 2004 is nao eleito handled

#Only result for 5203500 in 2012 is nao eleito Bom de Jesus
#Only result for 2803609 in 2016 is nao eleito Laranjeiras
#Only result for 3545506 in 2016 is nao eleito Sandovalina

In [151]:
vote.loc[len(vote.index)] = [2004, 1, 'eleicao ordinaria', 'SE','2802601','31518','prefeito', 'PP','11','88',np.nan, 'eleito', 2065]
vote.loc[len(vote.index)] = [2004, 1, 'eleicao ordinaria', 'SE', '2803609', '31712', 'prefeito', 'PTB', '14', '14', '119425', 'nao eleito', 5530]
vote.loc[len(vote.index)] = [2004, 1, 'eleicao ordinaria', 'SE', '2803609', '31712', 'prefeito', 'PFL', '25', '93', '1325512', 'eleito', 6609]

In [152]:
#vote[(vote.id_municipio == '2803609') & (vote.ano == 2016)]
#df[(df.ano == 2016) & (df.id_municipio == '2803609')]

In [153]:
df.shape

(100307, 27)

In [154]:
vote.loc[(vote.ano == 1996) | (vote.ano == 2000), 'sequencial_candidato'] = vote.loc[(vote.ano == 1996) | (vote.ano == 2000), 'sequencial_candidato'].fillna(vote.loc[(vote.ano == 1996) | (vote.ano == 2000), 'numero_candidato'])

df.loc[(df.ano == 1996) | (df.ano == 2000), 'sequencial'] = df.loc[(df.ano == 1996) | (df.ano == 2000), 'sequencial'].fillna(df.loc[(df.ano == 1996) | (df.ano == 2000), 'numero'])

In [155]:
df_turno = pd.merge( df, vote[['ano', 'id_municipio', 'sequencial_candidato', 'turno']], 
                how = 'left', right_on = ['ano', 'id_municipio', 'sequencial_candidato'], left_on = ['ano', 'id_municipio', 'sequencial'])

In [156]:
vote['annie_id'] = vote['ano'].astype(str) + vote['id_municipio'] + vote['sequencial_candidato'] + vote['turno'].astype(str)

df_turno['annie_id'] = df_turno['ano'].astype(str) + df_turno['id_municipio'] + df_turno['sequencial'] + df_turno['turno'].astype(str)
df_turno['annie_id'] = df_turno['annie_id'].apply(lambda x: x.split('.')[0])
#df['annie_id'] = df['ano'].astype(str) + df['id_municipio'] + df['numero'] 

In [157]:
vote.shape

(96841, 14)

In [158]:
vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,41,NaN,nao eleito,2388,19962704302411
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,16,NaN,nao eleito,1713,19961302603161
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,26,NaN,nao eleito,15721,19962927408261
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,29,NaN,nao eleito,705,19962927408291
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,30,NaN,nao eleito,989,19962927408301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96836,2020,2,eleicao ordinaria,CE,2303709,13730,prefeito,PROS,90,60000718891,441082,eleito,83588,20202303709600007188912
96837,2020,2,eleicao ordinaria,CE,2304400,13897,prefeito,PROS,90,60000653316,439843,nao eleito,624892,20202304400600006533162
96838,2004,1,eleicao ordinaria,SE,2802601,31518,prefeito,PP,11,88,NaN,eleito,2065,20042802601881
96839,2004,1,eleicao ordinaria,SE,2803609,31712,prefeito,PTB,14,14,119425,nao eleito,5530,20042803609141


In [159]:
#fill in nas from vote

#print(vote[(vote.id_candidato_bd.isna()) & (vote.ano > 2000)].shape[0])
cand_dir = pd.merge(vote1, df, how = 'left', left_on = ['id_municipio', 'ano', 'sequencial_candidato'], right_on = ['id_municipio', 'ano', 'sequencial'])
cand_dir['id_candidato_bd_full'] = cand_dir['id_candidato_bd_x']
cand_dir['id_candidato_bd_full'] = cand_dir['id_candidato_bd_full'].fillna(cand_dir['id_candidato_bd_y'])

cand_dir = cand_dir[['ano', 'id_municipio', 'sequencial_candidato', 'id_candidato_bd_full']]
cand_dir = cand_dir.drop_duplicates()

#print(cand_dir[cand_dir.id_candidato_bd_full.isna() & (cand_dir.ano > 2000)].shape[0])

vote = pd.merge(vote, cand_dir, how = 'left', on = ['id_municipio', 'ano', 'sequencial_candidato'])
df = pd.merge(df, cand_dir, how = 'left', left_on = ['id_municipio', 'ano', 'sequencial'], right_on = ['id_municipio', 'ano', 'sequencial_candidato'])


In [160]:
vote.shape

(96841, 15)

In [161]:
vote.columns = ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd_old', 'resultado', 'votos',
       'annie_id', 'id_candidato_bd_full']

vote.columns = ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd_old', 'resultado', 'votos',
       'annie_id', 'id_candidato_bd']

vote = vote[['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado', 'votos',
       'annie_id', 'id_candidato_bd_old']]


In [162]:
vote.shape

(96841, 15)

In [163]:
#vote = vote.drop_duplicates()

#df = df.drop_duplicates()

# vote = vote.drop_duplicates(['ano', 'id_municipio', 'numero_candidato', 'turno', 'tipo_eleicao'])
# df = df.drop_duplicates(['ano', 'id_municipio', 'numero'])

vote = vote.drop_duplicates(['ano', 'id_municipio', 'sequencial_candidato', 'turno', 'tipo_eleicao'])
# df = df.drop_duplicates(['ano', 'id_municipio', 'numero'])

In [164]:
vote.shape

(96841, 15)

In [165]:
#create variables that we're calculating below.

final = vote[['ano', 'id_municipio']].drop_duplicates().reset_index(drop = True)

final['runoff'] = np.zeros(final.shape[0])

final['winner_turno1'] = np.full([final.shape[0]], np.nan)
final['winner_turno2'] = np.full([final.shape[0]], np.nan)

final['totalvotes_turno1'] = np.zeros(final.shape[0])
final['totalvotes_turno2'] = np.zeros(final.shape[0])

final['winner_votes_turno_1'] = np.zeros(final.shape[0])
final['winner_votes_turno_2'] = np.zeros(final.shape[0])

final['margin_turno1'] = np.zeros(final.shape[0])
final['margin_turno2'] = np.zeros(final.shape[0])

final['incumbant'] = np.zeros(final.shape[0])
final['term_limited_seat'] = np.zeros(final.shape[0])

final['overall_winner'] = np.zeros(final.shape[0])

final['tipo'] = np.full([final.shape[0]], 'eleicao ordinaria')
final['special'] = np.zeros(final.shape[0])

final['overall_winner_numero_candidato'] = np.zeros(final.shape[0])




In [166]:
#Name of winner
#CPF of winner (if available)
#Name of Municipality

# Name of runner-up (second most voted candidate)

# CPF of runner-up (if available)
# Gender of runner-up
# Age of runner-up
# Occupation of runner-up
# Votes of runner up
# Proportion of votes runner up

In [167]:
# url_1996_1 = 'https://gist.githubusercontent.com/aulichney/88154f3d2e7bbf078032c5bede660cb0/raw/baa0bec7b690075e11c0765dc74575234e7e342a/resultado_1996_turno1.csv'
# url_1996_2 = 'https://gist.githubusercontent.com/aulichney/ec7bc2c9d454c2ae9850601e6ff46b43/raw/ee81de423ba17c36aad721d390df8617c4aa1ec1/resultado_1996_turno2.csv'

# url_2000_1 = 'https://gist.githubusercontent.com/aulichney/b0f10ec9cc24417d653792e01ac81719/raw/9835dad8f7aa06e4428ee0c0a404fbab2d132772/resultado_2000_turno1.csv'
# url_2000_2 = 'https://gist.githubusercontent.com/aulichney/25c535ec3a32e228dbfac973be5d64a8/raw/c43e819ae0c7ec11ecbf5d19185698c3fbb9270a/resultado_2000_turno2.csv'

# url_2004_1 = 'test'
# url_2004_2 = 'test'

url_1996_1 = f'/Users/annieulichney/Documents/GitHub/Deforestation/Political Data/ElectionData/1996_turno1.csv'
url_1996_2 = f'/Users/annieulichney/Documents/GitHub/Deforestation/Political Data/ElectionData/1996_turno2.csv'

url_2000_1 = f'/Users/annieulichney/Documents/GitHub/Deforestation/Political Data/ElectionData/2000_turno1.csv'
url_2000_2 = f'/Users/annieulichney/Documents/GitHub/Deforestation/Political Data/ElectionData/2000_turno2.csv'

url_2004_1 = f'/Users/annieulichney/Documents/GitHub/Deforestation/Political Data/ElectionData/2004_turno1.csv'
url_2004_2 = f'/Users/annieulichney/Documents/GitHub/Deforestation/Political Data/ElectionData/2004_turno2.csv'

df_1996_1 = pd.read_csv(url_1996_1).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_1996_2 = pd.read_csv(url_1996_2).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_1996 = pd.concat([df_1996_1, df_1996_2])

df_2000_1 = pd.read_csv(url_2000_1).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2000_2 = pd.read_csv(url_2000_2).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2000 = pd.concat([df_2000_1, df_2000_2])

df_2004_1 = pd.read_csv(url_2004_1).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2004_2 = pd.read_csv(url_2004_2).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2004 = pd.concat([df_2004_1, df_2004_2])



In [168]:
# url_1996_1 = 'https://gist.githubusercontent.com/aulichney/88154f3d2e7bbf078032c5bede660cb0/raw/baa0bec7b690075e11c0765dc74575234e7e342a/resultado_1996_turno1.csv'
# url_1996_2 = 'https://gist.githubusercontent.com/aulichney/ec7bc2c9d454c2ae9850601e6ff46b43/raw/ee81de423ba17c36aad721d390df8617c4aa1ec1/resultado_1996_turno2.csv'

# url_2000_1 = 'https://gist.githubusercontent.com/aulichney/b0f10ec9cc24417d653792e01ac81719/raw/9835dad8f7aa06e4428ee0c0a404fbab2d132772/resultado_2000_turno1.csv'
# url_2000_2 = 'https://gist.githubusercontent.com/aulichney/25c535ec3a32e228dbfac973be5d64a8/raw/c43e819ae0c7ec11ecbf5d19185698c3fbb9270a/resultado_2000_turno2.csv'

# year = 1996
# raw_turno1 = pd.read_csv(url_1996_1, header = 1)
# raw_turno1['turno'] = np.repeat(1, raw_turno1.shape[0])

# raw_turno2 = pd.read_csv(url_1996_2, header = 1)
# raw_turno2['turno'] = np.repeat(2, raw_turno2.shape[0])

# raw = pd.concat([raw_turno1, raw_turno2])
# raw.columns = ['cargo', 'sigla_uf', 'muni_name', 'sigla_partido', 'numero_candidato', 'cand_name',
#        'votos', 'resultado', 'pct', 'coligacao', 'drop1',
#        'turno']

# raw = raw[['cargo', 'sigla_uf', 'muni_name', 'sigla_partido', 'numero_candidato', 'cand_name',
#        'votos', 'resultado', 'pct', 'coligacao', 'turno']]

# raw['ano'] = year

# raw['muni_name'] = raw['muni_name'].apply(unidecode).apply(lambda x: x.lower())
# raw['cand_name'] = raw['cand_name'].apply(unidecode).apply(lambda x: x.lower())
# raw['resultado'] = raw['resultado'].apply(unidecode).apply(lambda x: x.lower())
# raw['cargo'] = raw['cargo'].apply(unidecode).apply(lambda x: x.lower())
# raw['votos'] = raw['votos'].apply(lambda x: x.replace(',', ''))
# raw['muni_name'] = raw['muni_name'].apply(lambda x: x.replace('\'', ''))
# raw['muni_name'] = raw['muni_name'].apply(lambda x: x.replace('-', ''))
# raw['muni_name'] = raw['muni_name'].apply(lambda x: x.replace(' ', ''))

# for col in raw.columns:
#     if type(raw[col][0]) == str:
#         raw[col] = raw[col].apply(unidecode).apply(lambda x: x.lower())

# raw['sigla_uf'] = raw['sigla_uf'].apply(unidecode).apply(lambda x: x.lower())

In [169]:
url_dir = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/diretorio.Rdata'
dir = pyreadr.read_r(url_dir)['df.diretorio']
dir = dir[['municipio', 'sigla_uf', 'id_municipio', 'id_municipio_6', 'id_municipio_tse', 'id_municipio_rf', 'id_municipio_bcb']]
dir.municipio = dir.municipio.apply(lambda x: x.replace('\'', ''))
dir.municipio = dir.municipio.apply(lambda x: x.replace('-', ''))
dir.municipio = dir.municipio.apply(lambda x: x.replace(' ', ''))

for col in dir.columns:
    if type(dir[col][0]) == str:
        dir[col] = dir[col].apply(unidecode).apply(lambda x: x.lower())
    
#raw_merged = pd.merge(raw, dir, how = 'left', left_on = ['sigla_uf', 'muni_name'], right_on = ['sigla_uf', 'municipio'])
#raw_merged[raw_merged.id_municipio_tse.isna()].shape

In [170]:
convert_muni = pd.read_csv('https://raw.githubusercontent.com/ekatovich/Municipality_Name_ID_crosswalk/master/Raw_Brazil_GeographicalUnits.csv', encoding='latin-1')
convert_muni = convert_muni[['uf', 'uf_name', 'mesoregion_stub', 'mesoregion_name',
       'microregion_stub', 'microregion_name', 'munic_stub', 'munic_code',
       'munic_name']]
convert_muni.munic_code = convert_muni.munic_code.astype(str)

for col in convert_muni.columns:
    if type(convert_muni[col][0]) == str:
        convert_muni[col] = convert_muni[col].apply(unidecode).apply(lambda x: x.lower())

convert_muni.munic_name = convert_muni.munic_name.apply(lambda x: x.replace('\'', ''))
convert_muni.munic_name = convert_muni.munic_name.apply(lambda x: x.replace('-', ''))
convert_muni.munic_name = convert_muni.munic_name.apply(lambda x: x.replace(' ', ''))
        
for col in convert_muni.columns:
    if type(convert_muni[col][0]) == str:
        convert_muni[col] = convert_muni[col].apply(unidecode).apply(lambda x: x.lower())

In [171]:
convert_muni = pd.read_csv('https://raw.githubusercontent.com/ekatovich/Municipality_Name_ID_crosswalk/master/Raw_Brazil_GeographicalUnits.csv', encoding='latin-1')
convert_muni = convert_muni[['uf', 'uf_name', 'mesoregion_stub', 'mesoregion_name',
       'microregion_stub', 'microregion_name', 'munic_stub', 'munic_code',
       'munic_name']]
convert_muni.munic_code = convert_muni.munic_code.astype(str)

df_only_1996 = df[(df.ano == 1996)]
vote_only_1996 = vote[(vote.resultado == 'eleito') & (vote.ano == 1996)]
winners_1996 = pd.merge(vote_only_1996, df_only_1996, how = 'left', left_on = ['ano', 'id_municipio', 'numero_candidato'], right_on = ['ano', 'id_municipio', 'numero'])

df_only_2000 = df[(df.ano == 2000)]
vote_only_2000 = vote[(vote.resultado == 'eleito') & (vote.ano == 2000)]
winners_2000 = pd.merge(vote_only_2000, df_only_2000, how = 'left', left_on = ['ano', 'id_municipio', 'numero_candidato'], right_on = ['ano', 'id_municipio', 'numero'])

df_only_2004 = df[(df.ano == 2004)]
vote_only_2004 = vote[(vote.resultado == 'eleito') & (vote.ano == 2004)]
winners_2004 = pd.merge(vote_only_2004, df_only_2004, how = 'left', left_on = ['ano', 'id_municipio', 'numero_candidato'], right_on = ['ano', 'id_municipio', 'numero'])

#add muni name to winners data
winners_1996 = pd.merge(winners_1996, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'id_municipio', right_on = 'munic_code')
winners_2000 = pd.merge(winners_2000, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'id_municipio', right_on = 'munic_code')
winners_2004 = pd.merge(winners_2004, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'id_municipio', right_on = 'munic_code')

#get just the winner names
df_1996 = df_1996[df_1996['Situação'] == 'Eleito']
df_1996 = df_1996[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_1996['Município'] = df_1996['Município'].apply(unidecode).apply(lambda x: x.lower())
winners_1996.munic_name = winners_1996.munic_name.apply(unidecode).apply(lambda x: x.lower())
df_1996['Candidato'] = df_1996['Candidato'].apply(unidecode).apply(lambda x: x.lower())

df_2000 = df_2000[df_2000['Situação'] == 'Eleito']
df_2000 = df_2000[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_2000['Município'] = df_2000['Município'].apply(unidecode).apply(lambda x: x.lower())
winners_2000.munic_name = winners_2000.munic_name.apply(unidecode).apply(lambda x: x.lower())
df_2000['Candidato'] = df_2000['Candidato'].apply(unidecode).apply(lambda x: x.lower())

df_2004_all = df_2004.copy()

df_2004 = df_2004[df_2004['Situação'] == 'Eleito']
df_2004 = df_2004[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_2004['Município'] = df_2004['Município'].apply(unidecode).apply(lambda x: x.lower())
winners_2004.munic_name = winners_2004.munic_name.apply(unidecode).apply(lambda x: x.lower())
df_2004['Candidato'] = df_2004['Candidato'].apply(unidecode).apply(lambda x: x.lower())

df_2004_all = df_2004_all[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_2004_all['Município'] = df_2004_all['Município'].apply(unidecode).apply(lambda x: x.lower())
df_2004_all['Candidato'] = df_2004_all['Candidato'].apply(unidecode).apply(lambda x: x.lower())

#get the names of the winners of all elections in 1996, 2000, 2004
winners_1996_names = pd.merge(winners_1996, df_1996, how = 'left', left_on = 'munic_name', right_on = 'Município')
winners_1996_names['won_1996'] = np.ones(winners_1996_names.shape[0])
winners_1996_subset = winners_1996_names[['id_municipio', 'munic_name', 'Candidato', 'won_1996']]

winners_2000_names = pd.merge(winners_2000, df_2000, how = 'left', left_on = 'munic_name', right_on = 'Município')
winners_2000_names['won_2000'] = np.ones(winners_2000_names.shape[0])
winners_2000_subset = winners_2000_names[['id_municipio', 'munic_name', 'Candidato', 'won_2000']]

winners_2004_names = pd.merge(winners_2004, df_2004, how = 'left', left_on = 'munic_name', right_on = 'Município')
winners_2004_names['won_2004'] = np.ones(winners_2004_names.shape[0])
winners_2004_subset = winners_2004_names[['id_municipio', 'munic_name', 'Candidato', 'won_2004']]


#1996 winners that also won in 2000
term_limited_2004 = pd.merge( winners_1996_subset, df_2000, how = 'inner', left_on = ['Candidato'], right_on = ['Candidato']).id_municipio

#2000 winners that also won in 2004
term_limited_2008 = pd.merge( winners_2000_subset, df_2004, how = 'inner', left_on = ['Candidato'], right_on = ['Candidato']).id_municipio

#incumbant in 2004 is someone that won in 2000 and ran in 2004
incumbant_2004 = pd.merge(winners_2000_subset, df_2004_all, how = 'inner', left_on = ['munic_name', 'Candidato'], right_on = ['Município', 'Candidato']).id_municipio

for muni in term_limited_2004:
    final.loc[final[(final.ano == 2004) & (final.id_municipio == muni)].index, 'term_limited_seat'] = 1

for muni in term_limited_2008:
    final.loc[final[(final.ano == 2008) & (final.id_municipio == muni)].index, 'term_limited_seat'] = 1

for muni in incumbant_2004:
    final.loc[final[(final.ano == 2004) & (final.id_municipio == muni)].index, 'incumbant'] = 1





In [172]:
for muni in term_limited_2004:
    final.loc[final[(final.ano == 2004) & (final.id_municipio == muni)].index, 'term_limited_seat'] = 1

for muni in term_limited_2008:
    final.loc[final[(final.ano == 2008) & (final.id_municipio == muni)].index, 'term_limited_seat'] = 1

for muni in incumbant_2004:
    final.loc[final[(final.ano == 2004) & (final.id_municipio == muni)].index, 'incumbant'] = 1

In [173]:
final = final.sort_values( by = ['ano'], axis = 0)

In [174]:
final = final.reset_index(drop = True)

In [175]:
vote.shape

(96841, 15)

In [176]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,41,NaN,nao eleito,2388,19962704302411,NaN
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,16,NaN,nao eleito,1713,19961302603161,NaN
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,26,NaN,nao eleito,15721,19962927408261,NaN
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,29,NaN,nao eleito,705,19962927408291,NaN
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,30,NaN,nao eleito,989,19962927408301,NaN


In [177]:
this_vote = vote[(vote.ano == 2004) & (vote.id_municipio == '4113700')]
winner_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]
winner_ordinary_turno1

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
20683,2004,1,eleicao ordinaria,PR,4113700,76678,prefeito,PT,13,412,69554,2º turno,72640,200441137004121,69554
46083,2004,1,eleicao ordinaria,PR,4113700,76678,prefeito,PSL,17,255,1366297,2º turno,85649,200441137002551,1366297


In [178]:
winner_ordinary_turno1

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
20683,2004,1,eleicao ordinaria,PR,4113700,76678,prefeito,PT,13,412,69554,2º turno,72640,200441137004121,69554
46083,2004,1,eleicao ordinaria,PR,4113700,76678,prefeito,PSL,17,255,1366297,2º turno,85649,200441137002551,1366297


In [179]:
runner_up_turno1 = winner_ordinary_turno1.sort_values(by = 'votos', ascending = False).reset_index(drop = True).iloc[1]
runner_up_turno1.numero_candidato

'13'

In [180]:
sum(winner_ordinary_turno1.votos)

158289

In [181]:

#Name of winner (merge) done
#CPF of winner (if available) (merge?) done
#Name of Municipality (merge)

# Name of runner-up (second most voted candidate) done

# CPF of runner-up (if available) done
# Gender of runner-up done
# Age of runner-up done
# Occupation of runner-up done
# Votes of runner up done
# Proportion of votes runner up calculate


#write over the type for this muni year which is the only 2 term special election type!
vote.loc[vote[(vote.id_municipio == '1600303')&(vote.ano == 2020)].index, 'tipo_eleicao'] = 'eleicao ordinaria'

#for i in tqdm(range(final.shape[0])): 
error_messages = []
missing_ano = []
missing_muni = []

for i in tqdm(range(final.shape[0])):  
#for i in [1500]:
    this_ano = final.iloc[i]['ano']
    this_muni = final.iloc[i]['id_municipio']

    did_runoff_occur = 0
    overall_winner = "None"

    if this_ano == 1996:
        continue
    if this_ano == 2000:
        continue

    this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]


    ##GET TURNO 1 WINNER AND MARGIN AND TOTAL VOTES

    #winner for ordinary election turno 1
    winner_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]
    #all_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria')  & (this_vote.turno == 1)]

    if winner_ordinary_turno1.shape[0] > 1: #more than one possible winner for turno 1 
        
        if '2º turno' in list(winner_ordinary_turno1.resultado): #find the max votes in turno 1 when 2º turno is result for two + people
            #runoff happened for this case
            final.loc[i, 'winner_turno1'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
            final.loc[i, 'winner_turno1_sequencial_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'sequencial_candidato']
            final.loc[i, 'winner_turno1_numero_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'numero_candidato']
            
            if len(winner_ordinary_turno1) > 1: #get runner up information, will be second most 2º turno person
                runner_up_turno1 = winner_ordinary_turno1.sort_values(by = 'votos', ascending = False).reset_index(drop = True).iloc[1]
                final.loc[i, 'runnerup_turno1'] = runner_up_turno1.id_candidato_bd
                final.loc[i, 'runnerup_turno1_sequencial_candidato'] = runner_up_turno1.sequencial_candidato
                final.loc[i, 'runnerup_turno1_numero_candidato'] = runner_up_turno1.numero_candidato
                final.loc[i, 'runnerup_turno1_annie_id'] = runner_up_turno1.annie_id
                final.loc[i, 'runnerup_turno1_votes'] = runner_up_turno1.votos

            #margin for exactly one winner for turno 1.
            number_of_votes = list(this_vote[this_vote.turno == 1].sort_values(by = 'votos', axis = 0, ascending = False).votos)
            if len(number_of_votes) == 1:
                final.loc[i, 'margin_turno1'] = number_of_votes[0] #margin
            else:
                final.loc[i, 'margin_turno1'] = number_of_votes[0] - number_of_votes[1] #margin
            
            final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
            final.loc[i, 'totalvotes_turno1'] = sum(number_of_votes) #total number of votes in turno 1

            did_runoff_occur = 1
        else: 
            error_messages.append("More than one turno 1 winner ordinary election for " + str(this_muni) + " " + str(this_ano))
    elif winner_ordinary_turno1.shape[0] == 0:  #missing winner for ordinary election! 
        #check if there's a special election when there's no ordinary election winner for turno 1.
        winner_other_type = vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)&(vote.resultado.isin(['eleito', 'eleito por qp']))]
        all_other_type = vote[((vote.ano == this_ano)&(vote.id_municipio == this_muni))]

        if list(winner_other_type.tipo_eleicao):
            final.loc[i, 'special'] = 1
            final.loc[i, 'winner_special'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
            final.loc[i, 'winner_special_sequencial_candidato'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'sequencial_candidato']
            final.loc[i, 'winner_special_numero_candidato'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'numero_candidato']

            outcome_type = np.unique(vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)].resultado)
            overall_winner = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
            overall_winner_annie_id = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'annie_id']
            overall_winner_sequencial_candidato = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'sequencial_candidato'] #overall_winner_sequencial_candidato
            overall_winner_numero_candidato = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'numero_candidato']

            final.loc[i, 'overall_winner'] = overall_winner
            final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
            final.loc[i, 'overall_winner_sequencial_candidato'] = overall_winner_sequencial_candidato
            final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato
            
            
            final.loc[i, 'tipo'] = list(winner_other_type.tipo_eleicao)[0]
            final.loc[i, 'outcomes'] = ", ".join(outcome_type)

            final.loc[i, 'special_votes'] = sum(winner_other_type.votos)

            if len(all_other_type) > 1: #get runner up information
                runner_up_special = all_other_type.sort_values(by = 'votos', ascending = False).reset_index(drop = True).iloc[1]
                final.loc[i, 'runnerup_special'] = runner_up_turno1.id_candidato_bd
                final.loc[i, 'runnerup_special_sequencial_candidato'] = runner_up_turno1.sequencial_candidato
                final.loc[i, 'runnerup_special_numero_candidato'] = runner_up_turno1.numero_candidato
                final.loc[i, 'runnerup_special_annie_id'] = runner_up_turno1.annie_id
                final.loc[i, 'runnerup_special_votes'] = runner_up_turno1.votos

            number_of_votes = list(vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)].sort_values(by = 'votos', axis = 0, ascending = False).votos)
            if winner_other_type.shape[0] == 1:
                if len(number_of_votes) == 1:
                    final.loc[i, 'margin_special'] = number_of_votes[0] #margin
                else:
                    final.loc[i, 'margin_special'] = number_of_votes[0] - number_of_votes[1] #margin
                
                final.loc[i, 'winner_votes_special'] = number_of_votes[0] #winner number of votes
                final.loc[i, 'totalvotes_special'] = sum(number_of_votes) #total number of votes in turno 1
            else:
                error_messages.append( str(winner_other_type.shape[0])  + 'special election winners for '+ str(this_muni) + " " + str(this_ano ) )
            
        else:
            error_messages.append("No winner turno 1 ordinary election for " + str(this_muni) + " " + str(this_ano ))
            missing_muni.append(this_muni)
            missing_ano.append(this_ano)

        this_vote[(this_vote.tipo_eleicao != 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

    else: #this is when there is exactly one winner in turno 1.

        final.loc[i, 'winner_turno1'] = list(winner_ordinary_turno1['id_candidato_bd'])[0]
        final.loc[i, 'winner_turno1_sequencial_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'sequencial_candidato']
        final.loc[i, 'winner_turno1_numero_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'numero_candidato']

        overall_winner = list(winner_ordinary_turno1.id_candidato_bd)[0]
        overall_winner_annie_id = list(winner_ordinary_turno1.annie_id)[0]
        overall_winner_sequencial_candidato = list(winner_ordinary_turno1.sequencial_candidato)[0]
        overall_winner_numero_candidato = list(winner_ordinary_turno1.numero_candidato)[0]

        final.loc[i, 'overall_winner'] = overall_winner
        final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
        final.loc[i, 'overall_winner_sequencial_candidato'] = overall_winner_sequencial_candidato
        final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato

        final.loc[i, 'turno1_votes'] = sum(this_vote.votos)
        if len(this_vote) > 1: #get runner up information
            runner_up_turno1 = this_vote.sort_values(by = 'votos', ascending = False).reset_index(drop = True).iloc[1]
            final.loc[i, 'runnerup_turno1'] = runner_up_turno1.id_candidato_bd
            final.loc[i, 'runnerup_turno1_sequencial_candidato'] = runner_up_turno1.sequencial_candidato
            final.loc[i, 'runnerup_turno1_numero_candidato'] = runner_up_turno1.numero_candidato
            final.loc[i, 'runnerup_turno1_annie_id'] = runner_up_turno1.annie_id
            final.loc[i, 'runnerup_turno1_votes'] = runner_up_turno1.votos

        #margin for exactly one winner for turno 1.
        number_of_votes = list(this_vote[this_vote.turno == 1].sort_values(by = 'votos', axis = 0, ascending = False).votos)
        if len(number_of_votes) == 1:
            final.loc[i, 'margin_turno1'] = number_of_votes[0] #margin
        else:
            final.loc[i, 'margin_turno1'] = number_of_votes[0] - number_of_votes[1] #margin
        final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
        final.loc[i, 'totalvotes_turno1'] = sum(number_of_votes) #total number of votes in turno 1
    

        #if '2º turno' in list(winner_ordinary_turno1.resultado):
        #    print("Not sure if this is supposed to happen:", i, this_ano, this_muni) 
        #   did_runoff_occur = 1
    

    ## GET TURNO 2 WINNER AND MARGIN AND TOTAL VOTES

    if did_runoff_occur == 1: 
        
        winner_ordinary_turno2 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', 'eleito por qp'])) & (this_vote.turno == 2)]
        all_turno2 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.turno == 2)]

        if winner_ordinary_turno2.shape[0] == 1:
            
            final.loc[i, 'winner_turno2'] = list(winner_ordinary_turno2.id_candidato_bd)[0]
            final.loc[i, 'winner_turno2_sequencial_candidato'] = list(winner_ordinary_turno2.sequencial_candidato)[0]
            final.loc[i, 'winner_turno2_numero_candidato'] = list(winner_ordinary_turno2.numero_candidato)[0]

            #use this for incumbant calculation
            overall_winner = list(winner_ordinary_turno2.id_candidato_bd)[0]
            overall_winner_annie_id = list(winner_ordinary_turno2.annie_id)[0]
            overall_winner_sequencial_candidato = list(winner_ordinary_turno1.sequencial_candidato)[0]
            overall_winner_numero_candidato = list(winner_ordinary_turno1.numero_candidato)[0]

            final.loc[i, 'overall_winner'] = overall_winner
            final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
            final.loc[i, 'overall_winner_sequencial_candidato'] = overall_winner_sequencial_candidato
            final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato

            if len(all_turno2) > 1: #get runner up information, will be second most 2º turno person
                runner_up_turno2 = all_turno2.sort_values(by = 'votos', ascending = False).reset_index(drop = True).iloc[1]
                final.loc[i, 'runnerup_turno2'] = runner_up_turno2.id_candidato_bd
                final.loc[i, 'runnerup_turno2_sequencial_candidato'] = runner_up_turno2.sequencial_candidato
                final.loc[i, 'runnerup_turno2_numero_candidato'] = runner_up_turno2.numero_candidato
                final.loc[i, 'runnerup_turno2_annie_id'] = runner_up_turno2.annie_id
                final.loc[i, 'runnerup_turno2_votes'] = runner_up_turno2.votos

            #margin for exactly one winner for turno 2.
            number_of_votes_2 = list(this_vote[this_vote.turno == 2].sort_values(by = 'votos', axis = 0, ascending = False).votos)
            if len(number_of_votes_2) == 1:
                final.loc[i, 'margin_turno2'] = number_of_votes_2[0] #margin
            else:
                final.loc[i, 'margin_turno2'] = number_of_votes_2[0] - number_of_votes_2[1] #margin
            final.loc[i, 'winner_votes_turno_2'] = number_of_votes_2[0] #winner number of votes
            final.loc[i, 'totalvotes_turno2'] = sum(number_of_votes_2) #total number of votes in turno 1
        elif winner_ordinary_turno2.shape[0] > 1: 
            error_messages.append('More than one turno 2 winner orginary election for' + str(this_muni) + " " + str(this_ano))
        elif winner_ordinary_turno2.shape[0] == 0 : 
            error_messages.append('No turno 2 winner orginary election for' + str(this_muni) + " " + str(this_ano))


    #INCUMBANT
    if this_ano >= 2004:
        this_vote_4yr = vote[(vote.ano == this_ano-4)&(vote.id_municipio == this_muni)]

        if overall_winner in list(this_vote_4yr.id_candidato_bd):
            final.loc[i, 'incumbant'] = 1
    
    

    
with open(r'errors.txt', 'w') as fp:
    for item in error_messages:
        fp.write("%s\n" % item)
    print('Done')





#rewrite the 2 term special election
final.loc[(final.ano == 2020)&(final.id_municipio == '1600303'), 'tipo'] = 'eleicoes municipais 2020 - ap'



    

100%|██████████| 33479/33479 [11:31<00:00, 48.43it/s] 

Done


In [182]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,outcomes,special_votes,runnerup_special,runnerup_special_sequencial_candidato,runnerup_special_numero_candidato,runnerup_special_annie_id,runnerup_special_votes,margin_special,winner_votes_special,totalvotes_special
0,1996,2704302,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,2201606,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,2202174,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,2202752,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,2203255,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
final.loc[final[final.tipo == 'eleicao ordinaria'].index, 'special'] = 0
final.loc[final[final.tipo != 'eleicao ordinaria'].index, 'special'] = 1

final.loc[final[final.runoff == 0].index, 'winner'] = final.loc[final[final.runoff == 0].index, 'winner_turno1']
final.loc[final[final.runoff == 1].index, 'winner'] = final.loc[final[final.runoff == 1].index, 'winner_turno2']
final.loc[final[final.special == 1].index, 'winner'] = final.loc[final[final.special == 1].index, 'winner_special']

final.loc[final[final.runoff == 0].index, 'margin'] = final.loc[final[final.runoff == 0].index, 'margin_turno1']
final.loc[final[final.runoff == 1].index, 'margin'] = final.loc[final[final.runoff == 1].index, 'margin_turno2']
final.loc[final[final.special == 1].index, 'margin'] = final.loc[final[final.special == 1].index, 'margin_special']

final.loc[final[final.runoff == 0].index, 'totalvotes'] = final.loc[final[final.runoff == 0].index, 'totalvotes_turno1']
final.loc[final[final.runoff == 1].index, 'totalvotes'] = final.loc[final[final.runoff == 1].index, 'totalvotes_turno2']
final.loc[final[final.special == 1].index, 'totalvotes'] = final.loc[final[final.special == 1].index, 'totalvotes_special']

final.loc[final[final.runoff == 0].index, 'sequencial_candidato'] = final.loc[final[final.runoff == 0].index, 'winner_turno1_sequencial_candidato']
final.loc[final[final.runoff == 1].index, 'sequencial_candidato'] = final.loc[final[final.runoff == 1].index, 'winner_turno2_sequencial_candidato']
final.loc[final[final.special == 1].index, 'sequencial_candidato'] = final.loc[final[final.special == 1].index, 'winner_special_sequencial_candidato']


final.loc[final[final.runoff == 0].index, 'numero_candidato'] = final.loc[final[final.runoff == 0].index, 'winner_turno1_numero_candidato']
final.loc[final[final.runoff == 1].index, 'numero_candidato'] = final.loc[final[final.runoff == 1].index, 'winner_turno2_numero_candidato']
final.loc[final[final.special == 1].index, 'numero_candidato'] = final.loc[final[final.special == 1].index, 'winner_special_numero_candidato']


final.loc[final[final.runoff == 0].index, 'winner_votes'] = final.loc[final[final.runoff == 0].index, 'winner_votes_turno_1']
final.loc[final[final.runoff == 1].index, 'winner_votes'] = final.loc[final[final.runoff == 1].index, 'winner_votes_turno_2']
final.loc[final[final.special == 1].index, 'winner_votes'] = final.loc[final[final.special == 1].index, 'winner_votes_special']

final.loc[final[final.runoff == 0].index, 'winner_votes_proportion'] = final.loc[final[final.runoff == 0].index, 'winner_votes_turno_1']/final.loc[final[final.runoff == 0].index, 'totalvotes_turno1']
final.loc[final[final.runoff == 1].index, 'winner_votes_proportion'] = final.loc[final[final.runoff == 0].index, 'winner_votes_turno_2']/final.loc[final[final.runoff == 1].index, 'totalvotes_turno2']
final.loc[final[final.special == 1].index, 'winner_votes_proportion'] = final.loc[final[final.special == 1].index, 'winner_votes_special']/final.loc[final[final.special == 1].index, 'totalvotes_special']


final.loc[final[final.runoff == 0].index, 'runnerup'] = final.loc[final[final.runoff == 0].index, 'runnerup_turno1']
final.loc[final[final.runoff == 1].index, 'runnerup'] = final.loc[final[final.runoff == 1].index, 'runnerup_turno2']
final.loc[final[final.special == 1].index, 'runnerup'] = final.loc[final[final.special == 1].index, 'runnerup_special']

final.loc[final[final.runoff == 0].index, 'runnerup_annie_id'] = final.loc[final[final.runoff == 0].index, 'runnerup_turno1_annie_id']
final.loc[final[final.runoff == 1].index, 'runnerup_annie_id'] = final.loc[final[final.runoff == 1].index, 'runnerup_turno2_annie_id']
final.loc[final[final.special == 1].index, 'runnerup_annie_id'] = final.loc[final[final.special == 1].index, 'runnerup_special_annie_id']

final.loc[final[final.runoff == 0].index, 'runnerup_sequencial'] = final.loc[final[final.runoff == 0].index, 'runnerup_turno1_sequencial_candidato']
final.loc[final[final.runoff == 1].index, 'runnerup_sequencial'] = final.loc[final[final.runoff == 0].index, 'runnerup_turno2_sequencial_candidato']
final.loc[final[final.special == 1].index, 'runnerup_sequencial'] = final.loc[final[final.special == 1].index, 'runnerup_special_sequencial_candidato']

final.loc[final[final.runoff == 0].index, 'runnerup_votes'] = final.loc[final[final.runoff == 0].index, 'runnerup_turno1_votes']
final.loc[final[final.runoff == 1].index, 'runnerup_votes'] = final.loc[final[final.runoff == 1].index, 'runnerup_turno2_votes']
final.loc[final[final.special == 1].index, 'runnerup_votes'] = final.loc[final[final.special == 1].index, 'runnerup_special_votes']

final.loc[final[final.runoff == 0].index, 'runnerup_votes_proportion'] = final.loc[final[final.runoff == 0].index, 'totalvotes_turno1']
final.loc[final[final.runoff == 1].index, 'runnerup_votes_proportion'] = final.loc[final[final.runoff == 1].index, 'totalvotes_turno2']
final.loc[final[final.special == 1].index, 'runnerup_votes_proportion'] = final.loc[final[final.special == 1].index, 'totalvotes_special']



In [184]:
#need overall runner up, where runoff is 1 it's the turno 1 1 ru

In [185]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,totalvotes,sequencial_candidato,numero_candidato,winner_votes,winner_votes_proportion,runnerup,runnerup_annie_id,runnerup_sequencial,runnerup_votes,runnerup_votes_proportion
0,1996,2704302,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
1,1996,2201606,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
2,1996,2202174,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
3,1996,2202752,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,1996,2203255,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [186]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,41,NaN,nao eleito,2388,19962704302411,NaN
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,16,NaN,nao eleito,1713,19961302603161,NaN
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,26,NaN,nao eleito,15721,19962927408261,NaN
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,29,NaN,nao eleito,705,19962927408291,NaN
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,30,NaN,nao eleito,989,19962927408301,NaN


In [187]:
remains_na = final[(final.ano > 2000 ) & (final.overall_winner_annie_id.isna()) & (final.ano  <= 2016 )]

remains_na = remains_na.reset_index()

error_messages2 = []
for i in range(remains_na.shape[0]):

    r_row = remains_na.iloc[i]


    r_vote = vote1[(vote1.ano == r_row.ano) & (vote1.id_municipio == r_row.id_municipio)]

    error_messages2.append('Only result for '+ r_row.id_municipio + ' in ' + str(r_row.ano) + ' is '  +  ", ".join(np.unique(r_vote.resultado)))

    with open(r'errors_final.txt', 'w') as fp:
        for item in error_messages2:
            fp.write("%s\n" % item)
        

In [188]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,totalvotes,sequencial_candidato,numero_candidato,winner_votes,winner_votes_proportion,runnerup,runnerup_annie_id,runnerup_sequencial,runnerup_votes,runnerup_votes_proportion
0,1996,2704302,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
1,1996,2201606,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
2,1996,2202174,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
3,1996,2202752,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,1996,2203255,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [189]:
#final = final.drop(['overall_winner_4yrs', 'overall_winner_8yrs'], axis = 1)

In [190]:
#double check on these ones to user sequencial candidato! 
final[((final.sequencial_candidato.isna()) & (final.ano > 2000) & (final.ano < 2020))]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,totalvotes,sequencial_candidato,numero_candidato,winner_votes,winner_votes_proportion,runnerup,runnerup_annie_id,runnerup_sequencial,runnerup_votes,runnerup_votes_proportion
6926,2004,3301009,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
17830,2012,5203500,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
24475,2016,2803609,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
26088,2016,3545506,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [191]:
final_4yrs = final.copy()
final_4yrs.ano = final_4yrs.ano + 4
final_4yrs = final_4yrs[['ano', 'id_municipio', 'overall_winner']]
final_4yrs.columns = ['ano', 'id_municipio', 'overall_winner_4yrs']

final_8yrs = final.copy()
final_8yrs.ano = final_8yrs.ano + 8
final_8yrs = final_8yrs[['ano', 'id_municipio', 'overall_winner']]
final_8yrs.columns = ['ano', 'id_municipio', 'overall_winner_8yrs']

final = pd.merge(final, final_4yrs, how = 'left', on = ['ano', 'id_municipio'])
final = pd.merge(final, final_8yrs, how = 'left', on = ['ano', 'id_municipio'])

final['term_limited_seat'] = 0


In [192]:
final[(~final.overall_winner_4yrs.isna()) & (final.overall_winner_4yrs == final.overall_winner_8yrs) & (final.overall_winner_4yrs != 0)]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,numero_candidato,winner_votes,winner_votes_proportion,runnerup,runnerup_annie_id,runnerup_sequencial,runnerup_votes,runnerup_votes_proportion,overall_winner_4yrs,overall_winner_8yrs
16789,2012,1718659,0.0,660519,NaN,1462.0,0.0,786.0,0.0,110.0,...,15,786.0,0.537620,97134,201217186592700000054331,270000005433,676.0,1462.0,1383914,1383914
16790,2012,1718709,0.0,1627991,NaN,1786.0,0.0,988.0,0.0,190.0,...,15,988.0,0.553191,74708,201217187092700000039921,270000003992,798.0,1786.0,1784479,1784479
16791,2012,1718865,0.0,1283689,NaN,4141.0,0.0,1907.0,0.0,385.0,...,40,1907.0,0.460517,662638,201217188652700000056351,270000005635,1522.0,4141.0,1611183,1611183
16794,2012,1718840,0.0,502812,NaN,2940.0,0.0,1322.0,0.0,282.0,...,22,1322.0,0.449660,716808,201217188402700000018701,270000001870,1040.0,2940.0,1083043,1083043
16799,2012,1716307,0.0,733079,NaN,2714.0,0.0,1593.0,0.0,472.0,...,15,1593.0,0.586957,437003,201217163072700000064891,270000006489,1121.0,2714.0,630398,630398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33462,2020,2514602,0.0,1022013,NaN,3109.0,0.0,2076.0,0.0,1273.0,...,10,2076.0,0.667739,174407,202025146021500008588251,150000858825,803.0,3109.0,1596245,1596245
33466,2020,2603207,0.0,605460,NaN,13499.0,0.0,9634.0,0.0,5769.0,...,10,9634.0,0.713682,606081,202026032071700007444811,170000744481,3865.0,13499.0,1437280,1437280
33474,2020,2516755,0.0,1220630,NaN,2306.0,0.0,1616.0,0.0,926.0,...,10,1616.0,0.700781,729743,202025167551500008675361,150000867536,690.0,2306.0,1796509,1796509
33475,2020,2516706,0.0,173648,NaN,9338.0,0.0,4677.0,0.0,727.0,...,12,4677.0,0.500857,905864,202025167061500008610151,150000861015,3950.0,9338.0,499416,499416


In [193]:
final[final.term_limited_seat == 1]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,numero_candidato,winner_votes,winner_votes_proportion,runnerup,runnerup_annie_id,runnerup_sequencial,runnerup_votes,runnerup_votes_proportion,overall_winner_4yrs,overall_winner_8yrs


In [194]:
final.loc[final[(~final.overall_winner_4yrs.isna()) & (final.overall_winner_4yrs == final.overall_winner_8yrs) & (final.overall_winner_4yrs != 0)].index, 'term_limited_seat'] = 1

In [195]:
sum(final.term_limited_seat == 1)
#used to be 6139

4691

In [196]:
final_cut = final[(final.ano > 2000) & (final.ano < 2020)]

In [197]:
final_cut.shape
#(22252, 36)

(22252, 61)

In [198]:
final_cut.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,numero_candidato,winner_votes,winner_votes_proportion,runnerup,runnerup_annie_id,runnerup_sequencial,runnerup_votes,runnerup_votes_proportion,overall_winner_4yrs,overall_winner_8yrs
5671,2004,2102200,0.0,1562413,NaN,11941.0,0.0,6582.0,0.0,1775.0,...,25,6582.0,0.551210,1698059,2004210220011,1,4807.0,11941.0,0.0,NaN
5672,2004,3203007,0.0,1588551,NaN,15620.0,0.0,10613.0,0.0,5606.0,...,23,10613.0,0.679449,95841,200432030072501,250,5007.0,15620.0,0.0,NaN
5673,2004,3203106,0.0,1302803,NaN,6986.0,0.0,4176.0,0.0,1366.0,...,23,4176.0,0.597767,555151,20043203106211,21,2810.0,6986.0,0.0,NaN
5674,2004,3203163,0.0,258030,NaN,7412.0,0.0,5146.0,0.0,2880.0,...,25,5146.0,0.694280,1247527,200432031631171,117,2266.0,7412.0,0.0,NaN
5675,2004,5214838,0.0,1773110,NaN,6027.0,0.0,2638.0,0.0,684.0,...,45,2638.0,0.437697,1539741,200452148383171,317,1954.0,6027.0,0.0,NaN


In [199]:
final_cut[final_cut.overall_winner_annie_id.isna()]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,numero_candidato,winner_votes,winner_votes_proportion,runnerup,runnerup_annie_id,runnerup_sequencial,runnerup_votes,runnerup_votes_proportion,overall_winner_4yrs,overall_winner_8yrs
17830,2012,5203500,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,1024004,123954
24475,2016,2803609,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,502673,119425
26088,2016,3545506,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,1313566,1313566


In [200]:


df_turno.instrucao = df_turno.instrucao.astype(str)


df_turno.loc[df_turno[df_turno.instrucao == 'analfabeto'].index, 'instrucao'] = 'illiterate'
df_turno.loc[df_turno[df_turno.instrucao == 'ensino fundamental completo'].index, 'instrucao'] = 'complete primary education'
df_turno.loc[df_turno[df_turno.instrucao == 'ensino fundamental incompleto'].index, 'instrucao'] = 'complete primary education'
df_turno.loc[df_turno[df_turno.instrucao == 'ensino medio completo'].index, 'instrucao'] = 'complete secondary education'
df_turno.loc[df_turno[df_turno.instrucao == 'ensino medio incompleto'].index, 'instrucao'] = 'incomplete secondary education'
df_turno.loc[df_turno[df_turno.instrucao == 'ensino superior completo'].index, 'instrucao'] = 'complete higher education'
df_turno.loc[df_turno[df_turno.instrucao == 'ensino superior incompleto'].index, 'instrucao'] = 'incomplete higher education'
df_turno.loc[df_turno[df_turno.instrucao == 'le e escreve'].index, 'instrucao'] = 'read and write'

print(np.unique(df_turno.instrucao))
df_turno.loc[df_turno[df_turno.instrucao == 'complete higher education'].index, 'completed_college'] = 1


['complete higher education' 'complete primary education'
 'complete secondary education' 'illiterate' 'incomplete higher education'
 'incomplete secondary education' 'nan' 'read and write']


In [201]:
demo_df = df_turno[['ano', 'sequencial', 'numero', 'id_municipio',
       'id_municipio_tse', 'id_candidato_bd', 'cpf', 'numero_partido', 'sigla_partido', 'ocupacao', 'municipio_nascimento', 'genero', 'instrucao', 'completed_college', 'nome', 'nome_urna', 'estado_civil', 'raca', 'idade', 'annie_id']]

In [202]:
df_turno.columns

Index(['ano', 'tipo_eleicao', 'sigla_uf', 'id_municipio', 'id_municipio_tse',
       'id_candidato_bd', 'cpf', 'titulo_eleitoral', 'sequencial', 'numero',
       'nome', 'nome_urna', 'numero_partido', 'sigla_partido', 'cargo',
       'situacao', 'ocupacao', 'data_nascimento', 'idade', 'genero',
       'instrucao', 'estado_civil', 'nacionalidade', 'sigla_uf_nascimento',
       'municipio_nascimento', 'email', 'raca', 'sequencial_candidato',
       'turno', 'annie_id', 'completed_college'],
      dtype='object')

In [230]:
merged = pd.merge(final_cut, demo_df, how = 'left', left_on = 'overall_winner_annie_id', right_on = 'annie_id')

In [231]:
merged.columns = ['ano', 'id_municipio'] + list(merged.columns[2:])

In [232]:
merged = merged[['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
       'totalvotes_turno1', 'totalvotes_turno2', 'winner_votes_turno_1',
       'winner_votes_turno_2', 'margin_turno1', 'margin_turno2', 'incumbant',
       'term_limited_seat', 'overall_winner', 'tipo', 'special',
       'overall_winner_numero_candidato', 'winner_turno1_sequencial_candidato',
       'winner_turno1_numero_candidato', 
       'overall_winner_sequencial_candidato', 'turno1_votes',
       'runnerup_turno1', 'runnerup_turno1_sequencial_candidato',
       'runnerup_turno1_numero_candidato', 
       'runnerup_turno1_votes', 'winner_turno2_sequencial_candidato',
       'winner_turno2_numero_candidato', 'runnerup_turno2',
       'runnerup_turno2_sequencial_candidato',
       'runnerup_turno2_numero_candidato', 
       'runnerup_turno2_votes', 'winner_special',
       'winner_special_sequencial_candidato',
       'winner_special_numero_candidato', 'outcomes', 'special_votes',
       'runnerup_special', 'runnerup_special_sequencial_candidato',
       'runnerup_special_numero_candidato', 'runnerup_annie_id',
       'runnerup_special_votes', 'margin_special', 'winner_votes_special',
       'totalvotes_special', 'winner', 'margin', 'totalvotes',
       'sequencial_candidato', 'numero_candidato', 'winner_votes',
       'winner_votes_proportion', 'runnerup', 
       'runnerup_sequencial', 'runnerup_votes', 'runnerup_votes_proportion', 'sequencial',
       'numero', 'id_municipio_tse', 'id_candidato_bd', 'cpf',
       'numero_partido', 'sigla_partido', 'ocupacao', 'municipio_nascimento',
       'genero', 'instrucao', 'completed_college', 'nome', 'nome_urna', 'estado_civil', 'raca',
       'idade']]

In [233]:
merged.columns = ['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
                'totalvotes_turno1', 'totalvotes_turno2', 'winner_votes_turno_1',
                'winner_votes_turno_2', 'margin_turno1', 'margin_turno2', 'incumbant',
                'term_limited_seat', 'overall_winner', 'tipo', 'special',
                'overall_winner_numero_candidato', 'winner_turno1_sequencial_candidato',
                'winner_turno1_numero_candidato', 'overall_winner_sequencial_candidato',
                'turno1_votes', 'runnerup_turno1',
                'runnerup_turno1_sequencial_candidato',
                'runnerup_turno1_numero_candidato', 'runnerup_turno1_votes',
                'winner_turno2_sequencial_candidato', 'winner_turno2_numero_candidato',
                'runnerup_turno2', 'runnerup_turno2_sequencial_candidato',
                'runnerup_turno2_numero_candidato', 'runnerup_turno2_votes',
                'winner_special', 'winner_special_sequencial_candidato',
                'winner_special_numero_candidato', 'outcomes', 'special_votes',
                'runnerup_special', 'runnerup_special_sequencial_candidato', 'runnerup_annie_id'
                'runnerup_special_numero_candidato', 'runnerup_annie_id', 'runnerup_special_votes',
                'margin_special', 'winner_votes_special', 'totalvotes_special',
                'winner', 'margin', 'totalvotes', 'sequencial_candidato',
                'numero_candidato', 'winner_votes', 'winner_votes_proportion',
                'runnerup', 'runnerup_sequencial', 'runnerup_votes',
                'runnerup_votes_proportion',  'overall_winner_sequencial', 'overall_winner_numero',
                'id_municipio_tse', 'overall_winner_id_candidato_bd', 'overall_winner_cpf', 'overall_winner_numero_partido',
                'overall_winner_sigla_partido', 'overall_winner_ocupacao', 'overall_winner_municipio_nascimento', 'overall_winner_genero',
                'overall_winner_instrucao', 'overall_winner_complete_college', 'overall_winner_nome', 'overall_winner_nome_urna', 'overall_winner_estado_civil', 'overall_winner_raca', 'overall_winner_idade']

In [234]:
demo_df_runnerup = df_turno[['cpf', 'numero_partido', 'sigla_partido', 'ocupacao', 'municipio_nascimento',
                             'genero', 'instrucao', 'completed_college', 'nome', 'nome_urna', 'estado_civil', 'raca', 'idade', 'annie_id']]

In [235]:
merged_runnerup = pd.merge(merged, demo_df_runnerup, how = 'left', left_on = 'runnerup_annie_id', right_on = 'annie_id')

In [236]:
merged_runnerup.columns

Index(['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
       'totalvotes_turno1', 'totalvotes_turno2', 'winner_votes_turno_1',
       'winner_votes_turno_2', 'margin_turno1', 'margin_turno2', 'incumbant',
       'term_limited_seat', 'overall_winner', 'tipo', 'special',
       'overall_winner_numero_candidato', 'winner_turno1_sequencial_candidato',
       'winner_turno1_numero_candidato', 'overall_winner_sequencial_candidato',
       'turno1_votes', 'runnerup_turno1',
       'runnerup_turno1_sequencial_candidato',
       'runnerup_turno1_numero_candidato', 'runnerup_turno1_votes',
       'winner_turno2_sequencial_candidato', 'winner_turno2_numero_candidato',
       'runnerup_turno2', 'runnerup_turno2_sequencial_candidato',
       'runnerup_turno2_numero_candidato', 'runnerup_turno2_votes',
       'winner_special', 'winner_special_sequencial_candidato',
       'winner_special_numero_candidato', 'outcomes', 'special_votes',
       'runnerup_special', 'runnerup_special_

In [237]:
merged_runnerup.columns = ['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
       'totalvotes_turno1', 'totalvotes_turno2', 'winner_votes_turno_1',
       'winner_votes_turno_2', 'margin_turno1', 'margin_turno2', 'incumbant',
       'term_limited_seat', 'overall_winner', 'tipo', 'special',
       'overall_winner_numero_candidato', 'winner_turno1_sequencial_candidato',
       'winner_turno1_numero_candidato', 'overall_winner_sequencial_candidato',
       'turno1_votes', 'runnerup_turno1',
       'runnerup_turno1_sequencial_candidato',
       'runnerup_turno1_numero_candidato', 'runnerup_turno1_votes',
       'winner_turno2_sequencial_candidato', 'winner_turno2_numero_candidato',
       'runnerup_turno2', 'runnerup_turno2_sequencial_candidato',
       'runnerup_turno2_numero_candidato', 'runnerup_turno2_votes',
       'winner_special', 'winner_special_sequencial_candidato',
       'winner_special_numero_candidato', 'outcomes', 'special_votes',
       'runnerup_special', 'runnerup_special_sequencial_candidato',
       'runnerup_annie_idrunnerup_special_numero_candidato',
       'runnerup_annie_id', 'runnerup_special_votes', 'margin_special',
       'winner_votes_special', 'totalvotes_special', 'winner', 'margin',
       'totalvotes', 'sequencial_candidato', 'numero_candidato',
       'winner_votes', 'winner_votes_proportion', 'runnerup',
       'runnerup_sequencial', 'runnerup_votes', 'runnerup_votes_proportion',
       'overall_winner_sequencial', 'overall_winner_numero',
       'id_municipio_tse', 'overall_winner_id_candidato_bd',
       'overall_winner_cpf', 'overall_winner_numero_partido',
       'overall_winner_sigla_partido', 'overall_winner_ocupacao',
       'overall_winner_municipio_nascimento', 'overall_winner_genero',
       'overall_winner_instrucao', 'overall_winner_complete_college', 'overall_winner_nome',
       'overall_winner_nome_urna', 'overall_winner_estado_civil',
       'overall_winner_raca', 'overall_winner_idade', 'runnerup_cpf', 'runnerup_numero_partido',
       'runnerup_sigla_partido', 'runnerup_ocupacao', 'runnerup_municipio_nascimento', 'runnerup_genero',
       'runnerup_instrucao', 'runnerup_complete_college', 'runnerup_nome', 'runnerup_nome_urna', 'runnerup_estado_civil', 'runnerup_raca', 'runnerup_idade',
       'annie_id']

In [238]:
merged_runnerup.runnerup_complete_college = merged_runnerup.runnerup_complete_college.fillna(0)
merged_runnerup.overall_winner_complete_college = merged_runnerup.overall_winner_complete_college.fillna(0)

In [239]:
merged_runnerup = merged_runnerup.drop('annie_id', axis = 1)


In [240]:
merged_runnerup.overall_winner_genero

0        masculino
1        masculino
2        masculino
3        masculino
4        masculino
           ...    
22247    masculino
22248    masculino
22249     feminino
22250    masculino
22251     feminino
Name: overall_winner_genero, Length: 22252, dtype: object

In [241]:
merged_runnerup.loc[merged_runnerup[merged_runnerup.overall_winner_genero == 'feminino'].index, 'overall_winner_feminino'] = 1
merged_runnerup.loc[merged_runnerup[merged_runnerup.runnerup_genero == 'feminino'].index, 'runnerup_feminino'] = 1

merged_runnerup.overall_winner_feminino = merged_runnerup.overall_winner_feminino.fillna(0)
merged_runnerup.runnerup_feminino = merged_runnerup.runnerup_feminino.fillna(0)

In [242]:
merged_runnerup[merged_runnerup.id_municipio == '1100023']

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,runnerup_genero,runnerup_instrucao,runnerup_complete_college,runnerup_nome,runnerup_nome_urna,runnerup_estado_civil,runnerup_raca,runnerup_idade,overall_winner_feminino,runnerup_feminino
1751,2004,1100023,0.0,988401,NaN,40434.0,0.0,20097.0,0.0,2361.0,...,feminino,complete higher education,1.0,Daniela Santana Amorim,Daniela Amorim,solteiro(a),NaN,29,0.0,1.0
9944,2008,1100023,0.0,988401,NaN,43041.0,0.0,31329.0,0.0,20207.0,...,masculino,complete primary education,0.0,Lorival Ribeiro De Amorim,"Lorival Amorim, Lorival Da Apa",divorciado(a),NaN,51,0.0,0.0
15417,2012,1100023,0.0,1624624,NaN,45818.0,0.0,20371.0,0.0,7565.0,...,masculino,complete secondary education,0.0,Saulo Pignaton,Saulo Da Daniela Amorim,solteiro(a),NaN,36,0.0,0.0
18580,2016,1100023,0.0,1584498,NaN,46141.0,0.0,26808.0,0.0,7475.0,...,masculino,complete secondary education,0.0,Lorival Ribeiro De Amorim,Lorival Amorim,divorciado(a),parda,59,0.0,0.0


In [243]:
url_dir = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/diretorio.Rdata'
dir = pyreadr.read_r(url_dir)['df.diretorio']
dir = dir[['municipio', 'id_municipio']]

dir.id_municipio = dir.id_municipio.astype(int)
merged_runnerup.id_municipio = merged_runnerup.id_municipio.astype(int)

In [244]:
merged_runnerup_name  = pd.merge(merged_runnerup, dir, how = 'left', left_on = 'id_municipio', right_on = 'id_municipio')

In [245]:
merged_runnerup_name[(merged_runnerup_name.ano == 2012) & (merged.totalvotes_turno2 != 0)].head(1)

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,runnerup_instrucao,runnerup_complete_college,runnerup_nome,runnerup_nome_urna,runnerup_estado_civil,runnerup_raca,runnerup_idade,overall_winner_feminino,runnerup_feminino,municipio
11390,2012,2927408,0.0,371244,371244,1292107.0,1341599.0,518976.0,717865.0,5626.0,...,complete higher education,1.0,Nelson Vicente Portela Pellegrino,Pelegrino,casado(a),NaN,52,0.0,0.0,Salvador


In [246]:
this_year = 2012
this_mun = '2927408'

vote1[(vote1.ano == this_year) & (vote1.id_municipio == this_mun)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id
2382,2012,1,eleicao ordinaria,BA,2927408,38490,prefeito,PRB,10,50000019361,395613,nao eleito,84094,20122927408500000193611
22812,2012,1,eleicao ordinaria,BA,2927408,38490,prefeito,PT,13,50000018141,1622611,2º turno,513350,20122927408500000181411
24401,2012,2,eleicao ordinaria,BA,2927408,38490,prefeito,PT,13,50000018141,1622611,nao eleito,623734,20122927408500000181412
41804,2012,1,eleicao ordinaria,BA,2927408,38490,prefeito,PMDB,15,50000003115,473984,nao eleito,121894,20122927408500000031151
64327,2012,1,eleicao ordinaria,BA,2927408,38490,prefeito,DEM,25,50000006657,371244,2º turno,518976,20122927408500000066571
65019,2012,2,eleicao ordinaria,BA,2927408,38490,prefeito,DEM,25,50000006657,371244,eleito,717865,20122927408500000066572
67075,2012,1,eleicao ordinaria,BA,2927408,38490,prefeito,PRTB,28,50000038559,1314975,nao eleito,20143,20122927408500000385591
88275,2012,1,eleicao ordinaria,BA,2927408,38490,prefeito,PSOL,50,50000019791,1617475,nao eleito,33650,20122927408500000197911


In [251]:
merged_runnerup_name = merged_runnerup_name.drop('runnerup_annie_idrunnerup_special_numero_candidato', axis = 1)
merged_runnerup_name.overall_winner_ocupacao = merged_runnerup_name.overall_winner_ocupacao.fillna(np.nan)
merged_runnerup_name.runnerup_ocupacao = merged_runnerup_name.runnerup_ocupacao.fillna(np.nan)

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,runnerup_instrucao,runnerup_complete_college,runnerup_nome,runnerup_nome_urna,runnerup_estado_civil,runnerup_raca,runnerup_idade,overall_winner_feminino,runnerup_feminino,municipio
0,2004,2102200,0.0,1562413,NaN,11941.0,0.0,6582.0,0.0,1775.0,...,complete secondary education,0.0,Patricia Magaldi Machado Jardim,Patricia Machado,casado(a),NaN,35,0.0,1.0,Buriti
1,2004,3203007,0.0,1588551,NaN,15620.0,0.0,10613.0,0.0,5606.0,...,complete secondary education,0.0,Gumercindo Gonçalves Vinand,Gumercino Vinand,casado(a),NaN,50,0.0,0.0,Iúna
2,2004,3203106,0.0,1302803,NaN,6986.0,0.0,4176.0,0.0,1366.0,...,complete secondary education,0.0,Gilmar Binotti,Bagu,casado(a),NaN,44,0.0,0.0,Jerônimo Monteiro
3,2004,3203163,0.0,258030,NaN,7412.0,0.0,5146.0,0.0,2880.0,...,complete primary education,0.0,Henrique Kefler Sobrinho,Dorico Kefler,casado(a),NaN,77,0.0,0.0,Laranja da Terra
4,2004,5214838,0.0,1773110,NaN,6027.0,0.0,2638.0,0.0,684.0,...,complete primary education,0.0,Isaias Nascimento,Isaias,casado(a),NaN,51,0.0,0.0,Nova Crixás


In [280]:
agriculture_jobs =  ['agricultor', 'agronomo','pecuarista','operador de implemento de agricultura pecuaria e exploracao florestal',
                    'produtor agropecuario', 'trabalhador florestal', 'trabalhador rural', 'tecnico em agronomia e agrimensura', 'pescador']

public_service_jobs = ['aposentado (exceto servidor publico)', 'agenciador de propaganda','cientista politico',
                        'diplomata','deputado','magistrado','membro das forcas armadas', 'membro do ministerio publico',
                        'militar reformado', 'ministro do poder judiciario e magistrado','policial civil', 'policial militar', 'prefeito',
                        'presidente da republica ministro de estado governador e prefeito', 'senador', 'senador deputado e vereador',
                        'servidor publico civil aposentado', 'servidor publico estadual',
                        'servidor publico federal', 'servidor publico municipal', 'serventuario de justica',
                        'vereador', 'assistente social', 'bombeiro militar']

health_jobs = ['biologo','biologo e biomedico', 'biomedico','enfermeiro','farmaceutico',
            'medico','operador de equipamento medico e odontologico','nutricionista e assemelhados','psicologo','protetico',
            'tecnico de enfermagem e assemelhados (exceto enfermeiro)', 'veterinario', 'agente de saude e sanitarista']

corporate_jobs = ['administrador', 'bancario e economiario', 'contador', 'economista', 'empresario','fiscal',
                    'diretor de empresas',]

law_jobs = ['advogado']

technical_jobs = ['engenheiro', 'estatistico', 'geologo','matematico e atuario', 
                    'operador de computador','programador de computador', 'analista de sistemas', 'estatistico',]

industrial_jobs = ['industrial','operador de instalacao de producao de energia eletrica e nuclear', 
                    'tecnico de mecanica', 'torneiro mecanico',
                    'trabalhador de construcao civil', 'tecnico de obras civis estradas saneamento e trabalhador assemelhado',
                    'trabalhador de fabricacao de papel e papelao', 'tecnico contabilidade estatistica economia domestica e administracao',
                    'trabalhador de fabricacao de artefatos de madeira', 'trabalhador metalurgico e siderurgico']

professional_jobs = ['bibliotecario', 'bibliotecario arquivista museologo e arqueologo', 'diretor de estabelecimento de ensino', 
                    'fonoaudiologo', 'historiador', 'odontologo', 'pedagogo', 'professor de ensino de primeiro e segundo graus', 
                    'professor de ensino fundamental', 'professor de ensino medio', 'professor de ensino superior', 
                    'professor e instrutor de formacao profissional', 'publicitario', 'quimico', 'relacoes-publicas', 'sociologo', 
                    'tecnico de laboratorio e raios x', 'tecnico de obras civis estradas saneamento e assemelhados', 
                    'tecnico de eletricidade eletronica e telecomunicacoes', 'tecnico de quimica', 'tecnico em edificacoes', 
                    'tecnico em informatica', 'tecnico textil', 'terapeuta']

mining_jobs = ['garimpeiro', 'tecnico de mineracao metalurgia e geologia', 'petroleiro', 'carvoeiro']

def job_category(job):
    if job in agriculture_jobs: return 'agriculture'
    elif job in public_service_jobs: return 'public service'
    elif job in health_jobs: return 'health'
    elif job in corporate_jobs: return 'corporate'
    elif job in law_jobs: return 'law'
    elif job in technical_jobs: return 'technical'
    elif job in professional_jobs: return 'professional'
    elif job in mining_jobs: return 'mining'
    else: return None

In [289]:
merged_runnerup_name['overall_winner_job_category'] = merged_runnerup_name.overall_winner_ocupacao.apply(job_category)
merged_runnerup_name['runnerup_job_category'] = merged_runnerup_name.runnerup_ocupacao.apply(job_category)

merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'agriculture'].index, 'overall_winner_agriculture_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'public service'].index, 'overall_winner_public_service_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'health'].index, 'overall_winner_health_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'corporate'].index, 'overall_winner_corporate_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'law'].index, 'overall_winner_law_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'technical'].index, 'overall_winner_technical_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'professional'].index, 'overall_winner_professional_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.overall_winner_job_category == 'mining'].index, 'overall_winner_mining_job'] = 1



merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'agriculture'].index, 'runnerup_agriculture_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'public service'].index, 'runnerup_public_service_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'health'].index, 'runnerup_health_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'corporate'].index, 'runnerup_corporate_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'law'].index, 'runnerup_law_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'technical'].index, 'runnerup_technical_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'professional'].index, 'runnerup_professional_job'] = 1
merged_runnerup_name.loc[merged_runnerup_name[merged_runnerup_name.runnerup_job_category == 'mining'].index, 'runnerup_mining_job'] = 1


merged_runnerup_name.overall_winner_agriculture_job = merged_runnerup_name.overall_winner_agriculture_job.fillna(0)
merged_runnerup_name.overall_winner_public_service_job = merged_runnerup_name.overall_winner_public_service_job.fillna(0)
merged_runnerup_name.overall_winner_health_job = merged_runnerup_name.overall_winner_health_job.fillna(0)
merged_runnerup_name.overall_winner_corporate_job = merged_runnerup_name.overall_winner_corporate_job.fillna(0)
merged_runnerup_name.overall_winner_law_job = merged_runnerup_name.overall_winner_law_job.fillna(0)
merged_runnerup_name.overall_winner_technical_job = merged_runnerup_name.overall_winner_technical_job.fillna(0)
merged_runnerup_name.overall_winner_professional_job = merged_runnerup_name.overall_winner_professional_job.fillna(0)
merged_runnerup_name.overall_winner_mining_job = merged_runnerup_name.overall_winner_mining_job.fillna(0)

merged_runnerup_name.runnerup_agriculture_job = merged_runnerup_name.runnerup_agriculture_job.fillna(0)
merged_runnerup_name.runnerup_public_service_job = merged_runnerup_name.runnerup_public_service_job.fillna(0)
merged_runnerup_name.runnerup_health_job = merged_runnerup_name.runnerup_health_job.fillna(0)
merged_runnerup_name.runnerup_corporate_job = merged_runnerup_name.runnerup_corporate_job.fillna(0)
merged_runnerup_name.runnerup_law_job = merged_runnerup_name.runnerup_law_job.fillna(0)
merged_runnerup_name.runnerup_technical_job = merged_runnerup_name.runnerup_technical_job.fillna(0)
merged_runnerup_name.runnerup_professional_job = merged_runnerup_name.runnerup_professional_job.fillna(0)
merged_runnerup_name.runnerup_mining_job = merged_runnerup_name.runnerup_mining_job.fillna(0)


In [307]:
fill_na_columns = ['winner_turno2', 'winner_turno2_sequencial_candidato', 'winner_turno2_numero_candidato',
                    'runnerup_turno2', 'runnerup_turno2_sequencial_candidato', 'runnerup_turno2_numero_candidato', 
                    'runnerup_turno2_votes', 'winner_special', 'winner_special_sequencial_candidato', 'winner_special_sequencial_candidato',
                    'outcomes', 'special_votes', 'runnerup_special', 'runnerup_special_sequencial_candidato', 'runnerup_special_votes', 
                    'margin_special', 'winner_votes_special', 'totalvotes_special']

for column in fill_na_columns:
    merged_runnerup_name[column] = merged_runnerup_name[column].fillna(np.nan)

In [ ]:
# indicator variables for the political party



#trim to only include what will be in the data used for the analysis. 

In [304]:
merged_runnerup_name.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,overall_winner_professional_job,overall_winner_mining_job,runnerup_agriculture_job,runnerup_public_service_job,runnerup_health_job,runnerup_corporate_job,runnerup_law_job,runnerup_technical_job,runnerup_professional_job,runnerup_mining_job
0,2004,2102200,0.0,1562413,NaN,11941.0,0.0,6582.0,0.0,1775.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2004,3203007,0.0,1588551,NaN,15620.0,0.0,10613.0,0.0,5606.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2004,3203106,0.0,1302803,NaN,6986.0,0.0,4176.0,0.0,1366.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2004,3203163,0.0,258030,NaN,7412.0,0.0,5146.0,0.0,2880.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2004,5214838,0.0,1773110,NaN,6027.0,0.0,2638.0,0.0,684.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [300]:
merged_runnerup_name.to_csv('PoliticalData.csv')

In [ ]:
#check the results
this_year = 2004
this_mun = '3203007'

merged[(merged.ano == this_year) & (merged.id_municipio == this_mun)].head(1)

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,margin_turno2,incumbant,term_limited_seat,overall_winner,tipo,special,overall_winner_numero_candidato,winner_turno1_sequencial_candidato,winner_turno1_numero_candidato,overall_winner_annie_id,overall_winner_sequencial_candidato,winner_turno2_sequencial_candidato,winner_turno2_numero_candidato,winner_special,winner_special_sequencial_candidato,winner_special_numero_candidato,outcomes,margin_special,totalvotes_special,winner,margin,totalvotes,sequencial_candidato,numero_candidato,overall_winner_4yrs,overall_winner_8yrs,ano_y,sequencial,numero,id_municipio_y,id_municipio_tse,id_candidato_bd,numero_partido,sigla_partido,ocupacao,municipio_nascimento,genero,instrucao,nome,nome_urna,estado_civil,raca,idade,annie_id
1,2004,3203007,0.0,1588551,NaN,15620.0,0.0,10613.0,0.0,5606.0,0.0,1.0,0,1588551,eleicao ordinaria,0.0,23,31,23,20043203007311,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1588551,5606.0,15620.0,31,23,0.0,NaN,2004.0,31,23,3203007,56596,1588551,23,PPS,engenheiro,Belo Horizonte,masculino,ensino superior completo,Rogerio Cruz Silva,Rogerio Cruz,casado(a),NaN,50,20043203007311


In [ ]:
dir[dir.id_municipio == 2103307]

,municipio,id_municipio
242,Codó,2103307
